In [ ]:

from pathlib import Path
import sys
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


plt.figure(figsize=(10, 30))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
# sns.set_context("paper")

# Configure the font to be serif, rather than sans
sns.set(font="serif")

sns.set(font_scale=1.5)
# plt.rcParams.update({'font.size': 15})

# look at that beautiful hack! Please help me find a better way to do this TODO
sys.path.append(str(Path(os.getcwd()).parent.parent))


from agentlab.analyze import plot_tools

save_dir = Path().home() / "dataset/ui-copilot/images"
save_dir.mkdir(parents=True, exist_ok=True)


%load_ext autoreload
%autoreload 2

### Load Masks

In [ ]:

masks_path =  "/Users/alexandre.lacoste/dataset/ui-copilot/MiniWoB++ task analysis - All tasks comparison.csv"

benchmark_names = [
    "WebGUM task (56)",
    "easy (auto)",
    "hard (auto)",
    "long context (auto)",
    # "easy",
    # "hard",
    # "rci-agent task (47)",
    # "long context",
    "pixel",
    "2d under standing",
    "x,y actions",
    "domain specific knowledge or exploration ?",
    # "memory",
    "long episode",
    # "rapid interaction",
    "action space limited",
    "requies augmented HTML and/or AXTree"
]

benchmark_masks = plot_tools.load_benchmark_masks(masks_path, benchmark_names=benchmark_names)

# merge columns
benchmark_masks["2D\nunderstanding"] = benchmark_masks["2D\nunderstanding"] | benchmark_masks["x, y\nactions"]
benchmark_masks = benchmark_masks.drop(columns=["x, y\nactions"])

sizes = [{"benchmar": benchmark, "count": benchmark_masks[benchmark].sum()} for benchmark in benchmark_masks.columns]
sizes = pd.DataFrame(sizes)
display(sizes)



### Load results massimo

In [ ]:
# csv_path = "/Users/alexandre.lacoste/dataset/ui-copilot/replan_agent_final/results.csv"
csv_path = Path().home() / "dataset/ui-copilot/results_massimo.csv"
df_mas = pd.read_csv(csv_path)

df_mas = plot_tools.modify_names(df_mas)

# filter code llama agent
df_mas = df_mas[df_mas.agent_model_name != "html_single_high-CodeLlama-13b"]

display(plot_tools.find_incomplete_exp(df_mas))
df_mas["cum_reward"] = df_mas["cum_reward"] + np.random.rand(len(df_mas)) * 0.001

# plot_tools.bootstrap_and_plot(plot_tools.add_benchmark_masks(df_mas, benchmark_masks), benchmark_masks.columns, fig_size=(15, 5), n_legend_rows=4)


### Load results rim

In [ ]:


csv_path = Path().home() / "dataset/ui-copilot/results_rim.csv"
df_rim = pd.read_csv(csv_path)

df_rim = plot_tools.modify_names(df_rim)
display(df_rim.columns)
display(plot_tools.find_incomplete_exp(df_rim, agent_columns_to_show=["max_steps"]))


df_rim["cum_reward"] = df_rim["cum_reward"] + np.random.rand(len(df_rim)) * 0.001


## effect of action space

In [ ]:

agent_order = [
    "html_code-gpt-4",
    "html_multi_high-gpt-4",
    # "tree_code-gpt-4",
    "html_code-gpt-3.5",
    "html_multi_both-gpt-3.5",
    "html_multi_high-gpt-3.5",
    # "html_multi_hybrid-gpt-3.5",
    "html_multi_low-gpt-3.5",
    "html_single_both-gpt-3.5",
    "html_single_high-gpt-3.5",
    # "html_single_hybrid-gpt-3.5",
    "html_single_low-gpt-3.5",
    # "tree_code-gpt-3.5",
    # "tree_multi_both-gpt-3.5",
    # "tree_multi_high-gpt-3.5",
    # "tree_multi_hybrid-gpt-3.5",
    # "tree_multi_low-gpt-3.5",
    # "tree_single_both-gpt-3.5",
    # "tree_single_high-gpt-3.5",
    # "tree_single_hybrid-gpt-3.5",
    # "tree_single_low-gpt-3.5",
]

markers = plot_tools.associate_action_markers(agent_order)

colors = sns.color_palette("colorblind", 5)
i = 0
agent_colors = {}
for agent in agent_order:
    if "multi" in agent:
        agent_colors[agent] = colors[2]
    elif "single" in agent: 
        agent_colors[agent] = colors[3]
    if "gpt-4" in agent:
        agent_colors[agent] = colors[0]
agent_colors["html_code-gpt-3.5"] = colors[1]

df_with_benchmarks = plot_tools.add_benchmark_masks(df_rim, benchmark_masks)

plot_tools.bootstrap_and_plot(df_with_benchmarks, benchmark_masks.columns, agent_order=agent_order, agent_colors=agent_colors,
                              agent_markers=markers, fig_size=(20,5), n_legend_rows=3)

plt.savefig(save_dir /  "action_space.pdf", bbox_inches='tight')



### Effect of model

In [ ]:
df = pd.concat([df_mas, df_rim])
# import json
# print(json.dumps(list(df["agent_model_name"].unique()), indent=4))

agent_order = [
    "html_code-gpt-4",
    "html_multi_high-gpt-4",
    "html_code-gpt-3.5",
    "html_multi_high-gpt-3.5",
    "html_code-CodeLlama-34b",
    "html_multi_high-CodeLlama-34b",
    "html_code-starchat",
    "html_multi_high-starchat",
    # "tree_multi_high-CodeLlama-34b",
    # "html_multi_low-CodeLlama-34b",
    # "html_single_high-CodeLlama-34b",
    # "tree_code-starchat",
    # "html_multi_low-starchat",
    # "html_single_high-starchat",
    # "tree_code-gpt-3.5",
    # "tree_multi_hybrid-gpt-3.5",
    # "tree_multi_both-gpt-3.5",
    # "html_multi_hybrid-gpt-3.5",
    # "html_multi_low-gpt-3.5",
    # "html_multi_both-gpt-3.5",
    # "tree_multi_high-gpt-3.5",
    # "tree_multi_low-gpt-3.5",
    # "html_single_both-gpt-3.5",
    # "html_single_high-gpt-3.5",
    # "tree_single_high-gpt-3.5",
    # "html_single_hybrid-gpt-3.5",
    # "html_single_low-gpt-3.5",
    # "tree_single_low-gpt-3.5",
    # "tree_single_both-gpt-3.5",
    # "tree_single_hybrid-gpt-3.5",
    # "tree_code-gpt-4"
]
# agent_order = plot_tools.sort_agents(agent_order, df)

model_order = ["gpt-4", "gpt-3.5", "CodeLlama-34b", "starchat"]
model_colors = dict( zip(model_order, sns.color_palette("colorblind")[:len(model_order)]))
# associate a color to each agent depending on model
agent_colors = {}
for agent in agent_order:
    for model in model_order:
        if model in agent:
            agent_colors[agent] = model_colors[model]
            break
    else:
        agent_colors[agent] = "black"

markers = plot_tools.associate_action_markers(agent_order)

df_with_benchmarks = plot_tools.add_benchmark_masks(df, benchmark_masks)

plot_tools.bootstrap_and_plot(df_with_benchmarks, benchmark_masks.columns, agent_order=agent_order, agent_colors=agent_colors,
                              agent_markers=markers, fig_size=(20,5), n_legend_rows=2)

plt.savefig(save_dir /  "model_effect.pdf", bbox_inches='tight')


## effect of obs space

In [ ]:
df = pd.concat([df_mas, df_rim])
# import json
# print(json.dumps(list(df["agent_model_name"].unique()), indent=4))

agent_order = [
    "html_code-gpt-4",
    "tree_code-gpt-4",
    # "html_multi_high-gpt-4",
    "html_code-gpt-3.5",
    "tree_code-gpt-3.5",
    # "html_multi_high-gpt-3.5",
    # "html_code-CodeLlama-34b",
    "html_multi_high-CodeLlama-34b",
    # "html_multi_high-starchat",
    "tree_multi_high-CodeLlama-34b",
    # "html_multi_low-CodeLlama-34b",
    # "html_single_high-CodeLlama-34b",
    "html_code-starchat",
    "tree_code-starchat",
    # "html_multi_low-starchat",
    # "html_single_high-starchat",
    # "tree_multi_hybrid-gpt-3.5",
    # "tree_multi_both-gpt-3.5",
    # "html_multi_hybrid-gpt-3.5",
    # "html_multi_low-gpt-3.5",
    # "html_multi_both-gpt-3.5",
    # "tree_multi_high-gpt-3.5",
    # "tree_multi_low-gpt-3.5",
    # "html_single_both-gpt-3.5",
    # "html_single_high-gpt-3.5",
    # "tree_single_high-gpt-3.5",
    # "html_single_hybrid-gpt-3.5",
    # "html_single_low-gpt-3.5",
    # "tree_single_low-gpt-3.5",
    # "tree_single_both-gpt-3.5",
    # "tree_single_hybrid-gpt-3.5",
]
# agent_order = plot_tools.sort_agents(agent_order, df)

model_order = ["gpt-4", "gpt-3.5", "CodeLlama-34b", "starchat"]
model_colors = dict( zip(model_order, sns.color_palette("colorblind")[:len(model_order)]))
# associate a color to each agent depending on model
agent_colors = {}
for agent in agent_order:
    for model in model_order:
        if model in agent:
            agent_colors[agent] = model_colors[model]
            break
    else:
        agent_colors[agent] = "black"

# markers = plot_tools.associate_markers(agent_order)

markers = {}
for agent in agent_order:
    if "tree" in agent:
        markers[agent] = "o"
    else:
        markers[agent] = "v"

df_with_benchmarks = plot_tools.add_benchmark_masks(df, benchmark_masks)

plot_tools.bootstrap_and_plot(df_with_benchmarks, benchmark_masks.columns, agent_order=agent_order, agent_colors=agent_colors,
                              agent_markers=markers, fig_size=(20,5), n_legend_rows=2)

plt.savefig(save_dir /  "obs_effect.pdf", bbox_inches='tight')

## Pivot table

In [ ]:

pivot_table = pd.pivot_table(df, values='cum_reward', columns='model_name', index='agent_name', aggfunc='mean')
display(pivot_table)
# save csv
pivot_table.to_csv(Path().home() / "dataset/ui-copilot/pivot_table.csv")

In [ ]:


benchmark_masks_ = benchmark_masks.rename(plot_tools.MINIWOB_CLASSNAME_TO_TASKNAME, axis='index')

plt.figure(figsize=(10, 40))
sns.heatmap(benchmark_masks_, annot=False, cmap="Blues", cbar=False, linewidths=.5)

# Rotate column names to be vertical and set font properties
plt.yticks(rotation=0, fontsize=10)

# Move the column labels to the top for better readability
plt.tick_params(axis="x", bottom=False, top=True, labelbottom=False, labeltop=True)
plt.xticks(rotation=30, fontsize=10)

plt.savefig(save_dir /  "benchmark_masks.pdf", bbox_inches='tight')



